In [1]:
import torch
import torch.nn as nn
import numpy as np

from PIL import Image

In [2]:
from torch.utils.data import DataLoader
from dataset import make_dataset
from setseed import set_seed

BATCH_SIZE = 32
RANDOM_SEED = 21
preprocess = None
augmentation = None
set_seed(RANDOM_SEED)

train_dataset = make_dataset(mode = 'train', RANDOM_SEED = RANDOM_SEED, preprocess = preprocess, augmentation=augmentation)
valid_dataset = make_dataset(mode = 'valid', RANDOM_SEED = RANDOM_SEED, preprocess = preprocess)
#test_dataset = make_dataset(mode = 'test', RANDOM_SEED = RANDOM_SEED, preprocess = preprocess)

train_loader = DataLoader(
    dataset=train_dataset, 
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=6,
    drop_last=True,
)

valid_loader = DataLoader(
    dataset=valid_dataset, 
    batch_size=2,
    shuffle=False,
    num_workers=2,
    drop_last=False
)

In [3]:
import segmentation_models_pytorch as smp

set_seed(RANDOM_SEED)

model = smp.UnetPlusPlus(
    encoder_name="tu-mobilenetv3_large_100",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=3,                      # model output channels (number of classes in your dataset)
)
model.name = 'Unet++_mobilenetv3_large_100'

In [4]:
LR = 1e-4
epoch = 15
VAL_EVERY = 1 # validation 주기

In [5]:
import torch.optim as optim

set_seed(RANDOM_SEED)

#optimizer = optim.Adam(params=model.parameters(), lr=LR, weight_decay=1e-6)

optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.01,
    momentum=0.9,
    weight_decay=1e-6,
    nesterov=True,
)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.2, patience=5, verbose=True
)

In [6]:
from loss import FocalLoss, DiceLoss, FocalTverskyLoss, IoULoss

set_seed(RANDOM_SEED)

#criterion = nn.BCEWithLogitsLoss()
#criterion = nn.SmoothL1Loss()
#criterion = [(nn.BCEWithLogitsLoss(), 0.75), (DiceLoss(), 0.25)]
#criterion = [(nn.BCEWithLogitsLoss(), 0.5), (nn.SmoothL1Loss(), 0.5)]
criterion = [(nn.BCEWithLogitsLoss(), 0.1), (FocalTverskyLoss(alpha = 0.7, beta = 0.3, gamma = 2), 0.15), (DiceLoss(), 0.5), (IoULoss(), 0.25)]

In [ ]:
from train import train

set_seed(RANDOM_SEED)
folder_name = train(model, train_loader, valid_loader, criterion, optimizer, epoch, VAL_EVERY)

Start training..
result model will be saved in /opt/ml/input/code/trained_model/last_model_2
2023-07-16 16:29:46 | Epoch [1/15], Step [0/3493]
2023-07-16 16:31:42 | Epoch [1/15], Step [150/3493], Loss: 0.60231
2023-07-16 16:33:34 | Epoch [1/15], Step [300/3493], Loss: 0.16827
2023-07-16 16:35:27 | Epoch [1/15], Step [450/3493], Loss: 0.149
2023-07-16 16:37:19 | Epoch [1/15], Step [600/3493], Loss: 0.14953
2023-07-16 16:39:12 | Epoch [1/15], Step [750/3493], Loss: 0.15267
2023-07-16 16:41:04 | Epoch [1/15], Step [900/3493], Loss: 0.14519
2023-07-16 16:42:56 | Epoch [1/15], Step [1050/3493], Loss: 0.12383
2023-07-16 16:44:49 | Epoch [1/15], Step [1200/3493], Loss: 0.11277
2023-07-16 16:46:42 | Epoch [1/15], Step [1350/3493], Loss: 0.11103
2023-07-16 16:48:34 | Epoch [1/15], Step [1500/3493], Loss: 0.12874
2023-07-16 16:50:27 | Epoch [1/15], Step [1650/3493], Loss: 0.11659
2023-07-16 16:52:19 | Epoch [1/15], Step [1800/3493], Loss: 0.10803
2023-07-16 16:54:12 | Epoch [1/15], Step [1950/34

  0%|          | 0/14957 [00:00<?, ?it/s]

iou per classes
sclera      : 0.70849
iris        : 0.86914
pupil       : 0.84418
mean dice: 0.80727

dice per classes
sclera      : 0.82502
iris        : 0.92860
pupil       : 0.91187
mean dice: 0.88850

new best pupil iou score: 0.84418
new best pupil dice score: 0.91187
new best mean iou score: 0.80727
new best mean dice score: 0.88850

2023-07-16 17:25:40 | Epoch [2/15], Step [0/3493]
2023-07-16 17:27:33 | Epoch [2/15], Step [150/3493], Loss: 0.0946
2023-07-16 17:29:25 | Epoch [2/15], Step [300/3493], Loss: 0.10293
2023-07-16 17:31:18 | Epoch [2/15], Step [450/3493], Loss: 0.10886
2023-07-16 17:33:10 | Epoch [2/15], Step [600/3493], Loss: 0.09847
2023-07-16 17:35:03 | Epoch [2/15], Step [750/3493], Loss: 0.10362
2023-07-16 17:36:55 | Epoch [2/15], Step [900/3493], Loss: 0.09488
2023-07-16 17:38:48 | Epoch [2/15], Step [1050/3493], Loss: 0.10014
2023-07-16 17:40:41 | Epoch [2/15], Step [1200/3493], Loss: 0.09633
2023-07-16 17:42:33 | Epoch [2/15], Step [1350/3493], Loss: 0.09517
202

  0%|          | 0/14957 [00:00<?, ?it/s]

iou per classes
sclera      : 0.71426
iris        : 0.87389
pupil       : 0.85206
mean dice: 0.81341

dice per classes
sclera      : 0.82861
iris        : 0.93144
pupil       : 0.91727
mean dice: 0.89244

new best pupil iou score: 0.85206
new best pupil dice score: 0.91727
new best mean iou score: 0.81341
new best mean dice score: 0.89244

2023-07-16 18:21:10 | Epoch [3/15], Step [0/3493]
2023-07-16 18:23:03 | Epoch [3/15], Step [150/3493], Loss: 0.08504
2023-07-16 18:24:55 | Epoch [3/15], Step [300/3493], Loss: 0.09456
2023-07-16 18:26:48 | Epoch [3/15], Step [450/3493], Loss: 0.09678
2023-07-16 18:28:41 | Epoch [3/15], Step [600/3493], Loss: 0.08866
2023-07-16 18:30:33 | Epoch [3/15], Step [750/3493], Loss: 0.09395
2023-07-16 18:32:26 | Epoch [3/15], Step [900/3493], Loss: 0.08615
2023-07-16 18:34:18 | Epoch [3/15], Step [1050/3493], Loss: 0.08778
2023-07-16 18:36:10 | Epoch [3/15], Step [1200/3493], Loss: 0.08595
2023-07-16 18:38:03 | Epoch [3/15], Step [1350/3493], Loss: 0.0848
202

...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



2023-07-17 04:49:05 | Epoch [14/15], Step [1650/3493], Loss: 0.06504
2023-07-17 04:50:58 | Epoch [14/15], Step [1800/3493], Loss: 0.05195
2023-07-17 04:52:51 | Epoch [14/15], Step [1950/3493], Loss: 0.05662
2023-07-17 04:54:43 | Epoch [14/15], Step [2100/3493], Loss: 0.05402
2023-07-17 04:56:36 | Epoch [14/15], Step [2250/3493], Loss: 0.05898
2023-07-17 04:58:28 | Epoch [14/15], Step [2400/3493], Loss: 0.05403
2023-07-17 05:00:21 | Epoch [14/15], Step [2550/3493], Loss: 0.0562
2023-07-17 05:02:13 | Epoch [14/15], Step [2700/3493], Loss: 0.05339
2023-07-17 05:04:06 | Epoch [14/15], Step [2850/3493], Loss: 0.06035
2023-07-17 05:05:59 | Epoch [14/15], Step [3000/3493], Loss: 0.06193
2023-07-17 05:07:52 | Epoch [14/15], Step [3150/3493], Loss: 0.05611
2023-07-17 05:09:44 | Epoch [14/15], Step [3300/3493], Loss: 0.05644
2023-07-17 05:11:37 | Epoch [14/15], Step [3450/3493], Loss: 0.05854
Start validation #14


  0%|          | 0/14957 [00:00<?, ?it/s]

iou per classes
sclera      : 0.74348
iris        : 0.88334
pupil       : 0.86028
mean dice: 0.82904

dice per classes
sclera      : 0.84696
iris        : 0.93656
pupil       : 0.92223
mean dice: 0.90192

new best pupil iou score: 0.86028
new best pupil dice score: 0.92223
new best mean iou score: 0.82904
best dice score was at 13

2023-07-17 05:23:01 | Epoch [15/15], Step [0/3493]
2023-07-17 05:24:54 | Epoch [15/15], Step [150/3493], Loss: 0.05515
2023-07-17 05:26:46 | Epoch [15/15], Step [300/3493], Loss: 0.06197
2023-07-17 05:28:39 | Epoch [15/15], Step [450/3493], Loss: 0.06059
2023-07-17 05:30:31 | Epoch [15/15], Step [600/3493], Loss: 0.05435
2023-07-17 05:32:24 | Epoch [15/15], Step [750/3493], Loss: 0.06281
2023-07-17 05:34:16 | Epoch [15/15], Step [900/3493], Loss: 0.05794
2023-07-17 05:36:09 | Epoch [15/15], Step [1050/3493], Loss: 0.05557
2023-07-17 05:38:01 | Epoch [15/15], Step [1200/3493], Loss: 0.05327
2023-07-17 05:39:54 | Epoch [15/15], Step [1350/3493], Loss: 0.05585


  0%|          | 0/14957 [00:00<?, ?it/s]

In [ ]:
import os
import cv2
import time
import torch
import numpy as np
from tqdm.auto import tqdm

In [ ]:
folder_name = '/opt/ml/input/code/trained_model/last_model'
model = torch.load(folder_name + '/best_pupil_dice.pt')
model.cuda()
model.eval()

def output_coloring(model, image_path):
    start = time.time()

    image = cv2.imread(image_path)
    image = np.array(image)
    image = np.pad(image, ((0,0),(0,16),(0,0)), 'constant', constant_values=0)

    # model에 넣기 위해 640 x 416으로 변환하고 batch 1로 만드는 과정
    img = image / 255.
    img = img.transpose(2, 0, 1)
    img = np.expand_dims(img, axis=0)
    img = torch.from_numpy(img).float()
    img = img.cuda()

    print("640 x 400 image padding까지 걸리는 시간: ", time.time() - start)

    start = time.time()
    output = model(img)
    print("model output까지 걸리는 시간: ", time.time() - start)

    # output에 threshold를 거쳐 최종적으로 예측한 클래스를 640 x 400 x 3으로 표기
    n = torch.sigmoid(output) # output 변환
    n = (n > 0.5).detach().cpu().numpy()
    n = np.squeeze(n, axis = 0)
    n = n.transpose(1, 2, 0)
    n = n.astype(np.uint8)

    # 원본 이미지에 예측 결과를 이용하여 색칠
    for i in range(640):
        for j in range(416):
            if n[i][j][0] == 1: # sclera, 공막, 적색
                image[i][j][0], image[i][j][1], image[i][j][2] = 255, 0, 0
            if n[i][j][1] == 1: # iris, 홍채, 녹색
                image[i][j][0], image[i][j][1], image[i][j][2] = 0, 255, 0
            if n[i][j][2] == 1: # pupil, 동공, 청색
                image[i][j][0], image[i][j][1], image[i][j][2] = 0, 0, 255
    image = Image.fromarray(image)
    # pillow의 image 객체 리턴
    return image

In [ ]:
def objdetect_inference(model, image_path):
    start = time.time()

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
    h, w, c = image.shape

    # 32에 가장 가까운 
    height = (h // 32) * 32
    width = (w // 32) * 32

    height = max(32, height)  # 32 pixel보다 작은 경우 32픽셀로 늘려주기
    width = max(32, width)
    
    image = cv2.resize(image, (width, height), interpolation=cv2.INTER_LANCZOS4)
    image = np.array(image)

    # model에 넣기 위해 640 x 416으로 변환하고 batch 1로 만드는 과정
    img = image / 255.
    img = img.transpose(2, 0, 1)
    img = np.expand_dims(img, axis=0)
    img = torch.from_numpy(img).float()
    img = img.cuda()

    #print(f"{h} x {w} image resize까지 걸리는 시간: ", time.time() - start)

    start = time.time()
    output = model(img)
    #print("model output까지 걸리는 시간: ", time.time() - start)

    # output에 threshold를 거쳐 최종적으로 예측한 클래스를 640 x 400 x 3으로 표기
    n = torch.sigmoid(output) # output 변환
    n = (n > 0.5).detach().cpu().numpy()
    n = np.squeeze(n, axis = 0)
    n = n.transpose(1, 2, 0)
    n = n.astype(np.uint8)

    # 원본 이미지에 예측 결과를 이용하여 색칠
    for i in range(height):
        for j in range(width):
            if n[i][j][0] == 1: # sclera, 공막, 적색
                image[i][j][0], image[i][j][1], image[i][j][2] = 255, 0, 0
            if n[i][j][1] == 1: # iris, 홍채, 녹색
                image[i][j][0], image[i][j][1], image[i][j][2] = 0, 255, 0
            if n[i][j][2] == 1: # pupil, 동공, 청색
                image[i][j][0], image[i][j][1], image[i][j][2] = 0, 0, 255
    image = Image.fromarray(image)
    # pillow의 image 객체 리턴
    return image

In [ ]:
path = '/opt/ml/input/code/objdet/train7'
output_path = '/opt/ml/input/code/inference_valid/'
image_num = 10

if not os.path.isdir(output_path):
    os.mkdir(output_path)

#for image_name in tqdm(os.listdir(path)[:image_num]):
for image_name in tqdm(os.listdir(path)):
    if image_name == '.ipynb_checkpoints':
        continue
    image_path = path + '/' + image_name
    image = objdetect_inference(model, image_path)
    
    Image.fromarray(cv2.imread(image_path)).save(output_path + image_name[:-4] + '_원본.png', 'png')
    image.save(output_path + image_name[:-4] + '_coloring.png', 'png')